<a href="https://colab.research.google.com/github/cemyyldz/Data-to-PDF-Report-Generator/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kullanılacak python paketlerini yükledim.


In [75]:
!pip install reportlab matplotlib pandas #Gerekli kütüphane kurulumları

İlk adım olan json dosyasını colabsa yükledim.Ardından verileri inceledim.


In [72]:
import json

# Json dosyasını okudum.Eğer dosyamız yoksa eksikse veyahutta boşsa bizi bilgilendirmesi için blok oluşturdum.Sonrasında verileri inceledim.

try:
    with open('data.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
except FileNotFoundError:
    print("❌ Hata: 'data.json' dosyası bulunamadı.")
    sys.exit()
except json.JSONDecodeError:
    print("❌ Hata: 'data.json' dosyası okunamadı veya geçersiz JSON formatında.")
    sys.exit()
print(json.dumps(data, indent=4, ensure_ascii=False))

{
    "rapor_basligi": "Aylık Satış Raporu",
    "donem": "Mart 2024 | Bölge Analizi",
    "summary": {
        "baslik": "Yönetici Özeti",
        "metin": "Bu rapor, şirketimizin Mart 2024 ayına ait satış performansını 5 farklı bölge bazında analiz etmektedir. Toplam satış tutarı, sipariş sayıları ve müşteri memnuniyeti oranları değerlendirilmiştir. Veriler ve trend analizi data.json dosyasındaki haftalık verilerle desteklenmiştir."
    },
    "bolgesel_satis_performansi": {
        "baslik": "Bölgesel Satış Performansı",
        "veriler": [
            {
                "bolge": "İstanbul",
                "satis": 156800,
                "siparis": 523,
                "ortalama": 300,
                "memnuniyet": 94
            },
            {
                "bolge": "Ankara",
                "satis": 124300,
                "siparis": 445,
                "ortalama": 279,
                "memnuniyet": 91
            },
            {
                "bolge": "İzmir",
         

2.adım olan rapor formatı için gerekli olan ayarları gerçekleştirdim.(açıklamalar yorum satırlarında mevcuttur)

In [73]:
import matplotlib.pyplot as plt
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import matplotlib
matplotlib.use('Agg')

# FONT AYARLARI (Türkçe karakter destekli).Bilgisayarımda bu font olmadığı için Colabsa ekledim

try:
    pdfmetrics.registerFont(TTFont('DejaVuSans', 'DejaVuSans.ttf'))
    pdfmetrics.registerFont(TTFont('DejaVuSans-Bold', 'DejaVuSans-Bold.ttf'))
    FONT_NAME = 'DejaVuSans'
    FONT_BOLD = 'DejaVuSans-Bold'
except Exception:
    print("❌ Hata: Font dosyaları bulunamadı. Lütfen 'DejaVuSans.ttf' ve 'DejaVuSans-Bold.ttf' dosyalarının kodla aynı dizinde olduğundan emin olun.")




# Haftalık Satış Grafiği


# Verileri kullanarak bir grafik oluşturdum ve başlık, renkler, eksenler gibi tüm görsel özelliklerini ayarladım.
haftalar = data['haftalik_satis_trendi']['veriler']['haftalar']
satislar = data['haftalik_satis_trendi']['veriler']['satislar']

fig, ax = plt.subplots(figsize=(7.5, 3.2))
ax.plot(haftalar, satislar, color='#4A90E2', linewidth=2.5, marker='o',
        markersize=7, markerfacecolor='#4A90E2', markeredgecolor='white', markeredgewidth=1.5)
ax.fill_between(haftalar, satislar, alpha=0.15, color='#4A90E2')
ax.set_title('Mart 2024 Haftalık Satış Gelişimi (₺)', fontsize=10, fontweight='normal',
             pad=12, color='#333333')
ax.grid(axis='y', linestyle='-', alpha=0.2, linewidth=0.8, color='#CCCCCC')
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('#DDDDDD')
ax.spines['bottom'].set_color('#DDDDDD')
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'₺{int(x/1000)}K'))
ax.tick_params(colors='#666666', labelsize=8)
ax.set_facecolor('#F8FBFF')
fig.patch.set_facecolor('white')
plt.tight_layout()


# Sonrasında tamamlanmış grafiği bir resim dosyası olarak kaydettim ve belleği temizledim.

plt.savefig('haftalik_satis_grafik.png', dpi=120, bbox_inches='tight', facecolor='white')
plt.close()



3.adım olan pdf oluşturma aşamasına geçiş yaptım.(açıklamalar yorum satırlarında mevcuttur)

In [74]:
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.lib.enums import TA_LEFT, TA_CENTER, TA_JUSTIFY
import matplotlib
matplotlib.use('Agg')
import pandas as pd
import sys

# PDF OLUŞTURMA
# ---------------------------
#PDF dokümanının temel yapısını (sayfa boyutu, kenar boşlukları) ve raporda kullanılacak tüm yazı stillerini tanımladım.
pdf = SimpleDocTemplate(
    'report.pdf',
    pagesize=A4,
    rightMargin=45, leftMargin=45,
    topMargin=40, bottomMargin=40
)

FULL_WIDTH = 505
elemanlar = []

styles = getSampleStyleSheet()
styles.add(ParagraphStyle(name='BoxTitle', fontSize=11, spaceAfter=6, fontName=FONT_BOLD, textColor=colors.HexColor('#1A1A1A')))
styles.add(ParagraphStyle(name='BoxedBody', fontSize=8.5, leading=12, alignment=TA_JUSTIFY, fontName=FONT_NAME, textColor=colors.HexColor('#333333'), leftIndent=8, rightIndent=8, spaceBefore=6, spaceAfter=6))
styles.add(ParagraphStyle(name='MainTitle', fontSize=18, alignment=TA_CENTER, spaceAfter=12, fontName=FONT_BOLD, textColor=colors.HexColor('#1A1A1A')))
styles.add(ParagraphStyle(name='SubTitle', fontSize=9, alignment=TA_CENTER, textColor=colors.HexColor('#666666'), spaceAfter=10, fontName=FONT_NAME))


styles.add(ParagraphStyle(
    name='SectionTitle',
    fontSize=11,
    spaceAfter=6,
    fontName=FONT_BOLD,
    textColor=colors.HexColor('#1A1A1A'),
    leftIndent=0,
    alignment=TA_LEFT
))

styles.add(ParagraphStyle(
    name='BulletStyle',
    parent=styles['Normal'],
    fontName=FONT_NAME,
    fontSize=8.5,
    leading=14, # Satır arası boşluğu
    leftIndent=12, # Madde işaretinin soldan girintisi
    textColor=colors.HexColor('#3D3D3D')
))

styles.add(ParagraphStyle(name='FooterText', fontSize=7.5, alignment=TA_CENTER, textColor=colors.HexColor('#999999'), spaceBefore=12, fontName=FONT_NAME))

#Tanımlanan stilleri kullanarak raporun başlık ve çizgi gibi ilk içerik elemanlarını oluşturdum.Sonrasında rapora ekledim

elemanlar.append(Paragraph("<b>Aylık Satış Raporu</b>", styles['MainTitle']))
elemanlar.append(Paragraph("Mart 2024 | Bölge Analizi", styles['SubTitle']))

header_line = Table([['']], colWidths=[FULL_WIDTH], rowHeights=[1])
header_line.setStyle(TableStyle([('LINEBELOW', (0,0), (-1,-1), 1, colors.black)]))
elemanlar.append(header_line)
elemanlar.append(Spacer(1, 12))

#Yönetici Özeti bölümünü, arka planı, iç boşlukları ve kenar çizgisi olan stilli bir kutu olarak oluşturdum ve rapora ekledim.
summary_content = Paragraph("<b>Yönetici Özeti</b>", styles['BoxTitle'])
summary_text = Paragraph(data['summary']['metin'], styles['BoxedBody'])
box_table = Table([[summary_content], [summary_text]], colWidths=[FULL_WIDTH])
box_table.setStyle(TableStyle([
    ('BACKGROUND', (0,0), (-1,-1), colors.HexColor('#F0F7FF')),
    ('LEFTPADDING', (0,0), (-1,-1), 12), ('RIGHTPADDING', (0,0), (-1,-1), 12),
    ('TOPPADDING', (0,0), (-1,0), 12), ('BOTTOMPADDING', (0,0), (-1,0), 6),
    ('TOPPADDING', (0,1), (-1,-1), 4), ('BOTTOMPADDING', (0,1), (-1,-1), 12),
    ('LINEBEFORE', (0,0), (0,-1), 3, colors.HexColor('#4A90E2')),
]))
elemanlar.append(box_table)
elemanlar.append(Spacer(1, 12))

# Bölgesel Satış Performansı
# Pandas ile satış verilerini hazırlayıp formatladım, ardından bu verileri kullanarak ReportLab'de stilli bir tablo oluşturdum sonrasında rapora ekledim.

elemanlar.append(Paragraph("<b>Bölgesel Satış Performansı</b>", styles['SectionTitle']))
section_line = Table([['']], colWidths=[FULL_WIDTH], rowHeights=[1])
section_line.setStyle(TableStyle([('LINEBELOW', (0,0), (-1,-1), 0.5, colors.HexColor('#CCCCCC'))]))
elemanlar.append(section_line)
elemanlar.append(Spacer(1, 8))

df_bolge = pd.DataFrame(data['bolgesel_satis_performansi']['veriler'])
df_bolge['Satış (₺)'] = df_bolge['satis'].apply(lambda x: f"{x:,}".replace(',', '.'))
df_bolge['Sipariş'] = df_bolge['siparis'].astype(str)
df_bolge['Ortalama (₺)'] = df_bolge['ortalama'].astype(str)
df_bolge['Memnuniyet (%)'] = df_bolge['memnuniyet'].astype(str)
df_bolge = df_bolge.rename(columns={'bolge':'Bölge'})
df_bolge = df_bolge[['Bölge','Satış (₺)','Sipariş','Ortalama (₺)','Memnuniyet (%)']]
tablo_veri = [df_bolge.columns.tolist()] + df_bolge.values.tolist()

tablo = Table(tablo_veri, colWidths=[105, 100, 80, 100, 120])
tablo_style = [
    ('BACKGROUND',(0,0),(-1,0),colors.HexColor('#EAF2FF')),
    ('TEXTCOLOR',(0,0),(-1,0),colors.HexColor('#1A1A1A')),
    ('FONTNAME',(0,0),(-1,0),FONT_BOLD),
    ('FONTSIZE',(0,0),(-1,0),9),
    ('ALIGN',(0,0),(-1,0),'CENTER'),
    ('BOTTOMPADDING',(0,0),(-1,0),7),
    ('TOPPADDING',(0,0),(-1,0),7),
    ('FONTNAME',(0,1),(-1,-1),FONT_NAME),
    ('FONTSIZE',(0,1),(-1,-1),8.5),
    ('TEXTCOLOR',(0,1),(-1,-1),colors.HexColor('#333333')),
    ('VALIGN',(0,1),(-1,-1),'MIDDLE'),
    ('ALIGN',(1,1),(-1,-1),'RIGHT'),
    ('ALIGN',(0,1),(0,-1),'LEFT'),
    ('LEFTPADDING',(0,0),(-1,-1),6),
    ('RIGHTPADDING',(0,0),(-1,-1),6),
    ('TOPPADDING',(0,1),(-1,-1),5),
    ('BOTTOMPADDING',(0,1),(-1,-1),5),
    ('LINEBELOW',(0,0),(-1,0),1,colors.HexColor('#D0D0D0')),
    ('INNERGRID',(0,1),(-1,-1),0.25,colors.HexColor('#E0E0E0')),
    ('BOX',(0,0),(-1,-1),0.75,colors.HexColor('#CCCCCC')),
]
for i in range(1,len(tablo_veri)):
    if i % 2 == 0:
        tablo_style.append(('BACKGROUND',(0,i),(-1,i),colors.HexColor('#F7FAFF')))
    else:
        tablo_style.append(('BACKGROUND',(0,i),(-1,i),colors.whitesmoke))
tablo.setStyle(TableStyle(tablo_style))
elemanlar.append(tablo)
elemanlar.append(Spacer(1, 12))

# Haftalık Satış Trendi başlığını ve altındaki ayracı ekledim, ardından daha önce oluşturulmuş satış grafiği görselini rapora dahil ettim.

elemanlar.append(Paragraph("<b>Haftalık Satış Trendi</b>", styles['SectionTitle']))
section_line2 = Table([['']], colWidths=[FULL_WIDTH], rowHeights=[1])
section_line2.setStyle(TableStyle([('LINEBELOW', (0,0), (-1,-1), 0.5, colors.HexColor('#CCCCCC'))]))
elemanlar.append(section_line2)
elemanlar.append(Spacer(1, 8))

elemanlar.append(Image('haftalik_satis_grafik.png', width=FULL_WIDTH, height=200))
elemanlar.append(Spacer(1, 12))

#Temel Bulgular verilerini bir döngü içinde işleyerek madde madde formatla, bu maddeleri stilli bir kutu (arka plan, boşluklar) içine yerleştirdim ve rapora ekledim.

bulgu_flowables = []
for madde in data['temel_bulgular']['maddeler']:

    text = f'• <font name="{FONT_BOLD}">{madde["madde"]}:</font> {madde["detay"]}'
    bulgu_flowables.append(Paragraph(text, styles['BulletStyle']))


styles['BoxTitle'].leftIndent = 0
bulgular_title = Paragraph("Temel Bulgular", styles['BoxTitle'])


content_list = [[bulgular_title]] + [[p] for p in bulgu_flowables]

bulgular_box = Table(content_list, colWidths=[FULL_WIDTH])

bulgular_box.setStyle(TableStyle([

    ('BACKGROUND', (0,0), (-1,-1), colors.HexColor('#FFF9E6')),


    ('LEFTPADDING', (0,0), (-1,-1), 18),
    ('RIGHTPADDING', (0,0), (-1,-1), 18),
    ('TOPPADDING', (0,0), (-1,-1), 16),
    ('BOTTOMPADDING', (0,0), (-1,-1), 16),


    ('BOTTOMPADDING', (0,0), (0,0), 12),


    ('TOPPADDING', (0,1), (0,-1), 5),
    ('BOTTOMPADDING', (0,1), (0,-1), 5),
]))

elemanlar.append(bulgular_box)
elemanlar.append(Spacer(1, 12))


# Raporun en sonuna, ayırıcı bir çizgi ve altında yer alan footer metnini ekledim.
footer_line = Table([['']], colWidths=[FULL_WIDTH], rowHeights=[1])
footer_line.setStyle(TableStyle([('LINEBELOW', (0,0), (-1,-1), 0.5, colors.HexColor('#CCCCCC'))]))
elemanlar.append(footer_line)
elemanlar.append(Spacer(1, 8))
elemanlar.append(Paragraph("Hazırlayan: Satış Analizi Ekibi | Rapor Kodu: SAT-2024-03", styles['FooterText']))

# Son olarak pdfi oluşturdum.
pdf.build(elemanlar)
print("✅ report.pdf başarıyla oluşturuldu.")

✅ report.pdf başarıyla oluşturuldu.
